<a href="https://colab.research.google.com/github/AtSourav/AE-n-VAE-with-CB-loss-on-stl10/blob/main/AE_sifar10_mse_HeNormal_L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow import math
import tensorflow_datasets as tfds
from keras import losses
from keras import layers
from keras import utils
from keras import backend as K
from keras import initializers

import os
import random
import matplotlib.pyplot as plt

In [2]:
img_ht = 32
img_wd = 32

input_size = (img_ht,img_wd,3)
latent_dim = 200
batch_size = 128

initializer = initializers.HeNormal(seed=100)

In [3]:
ds = tfds.load('cifar10', split='train')
ds2= tfds.load('cifar10', split='test')

In [4]:
ds_to_np = list(ds.as_numpy_iterator())
ds2_to_np = list(ds2.as_numpy_iterator())

In [5]:
img_train = np.array([x['image'] for x in ds_to_np])
img_valid = np.array([x['image'] for x in ds2_to_np])

img_train = img_train/255
img_valid = img_valid/255

In [6]:
encoder_input = keras.Input(shape=input_size)

x = layers.Conv2D(64, 3, padding="valid", kernel_initializer=initializer)(encoder_input)
x = layers.ReLU()(x)

x = layers.Conv2D(64, 2, padding="valid", kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(x)

x = layers.Conv2D(128, 3, padding="valid", kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

#x = layers.Conv2D(128, 2, padding="valid", kernel_initializer=initializer)(x)
#x = layers.ReLU()(x)

x = layers.Conv2D(128, 2, padding="valid", kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2D(128, 2, padding="valid", kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2D(256, 2, padding="valid", kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2D(256, 2, padding="valid", kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2D(256, 2, padding="valid", kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2D(256, 2, padding="valid", kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2D(512, 2, padding="valid", kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2D(512, 2, padding="valid", kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2D(512, 2, padding="valid", kernel_initializer=initializer)(x)
#x = layers.BatchNormalization(axis=-1)(x)
x = layers.ReLU()(x)

x = layers.Conv2D(512, 2, padding="valid", kernel_initializer=initializer)(x)
#x = layers.BatchNormalization(axis=-1)(x)
x = layers.ReLU()(x)

x = layers.Conv2D(512, 1, padding="valid", kernel_initializer=initializer)(x)
#x = layers.BatchNormalization(axis=-1)(x)
x = layers.ReLU()(x)

x = layers.Conv2D(1024, 1, padding="valid", kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Flatten()(x)

x = layers.Dense(4*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(2*latent_dim)(x)
x = layers.ReLU()(x)


z = layers.Dense(latent_dim, name="z")(x)

encoder = keras.Model(encoder_input, z, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 64)        1792      
                                                                 
 re_lu (ReLU)                (None, 30, 30, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        16448     
                                                                 
 re_lu_1 (ReLU)              (None, 29, 29, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 64)        0         
 D)                                                              
                                                           

In [7]:
latent_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(2*latent_dim)(latent_input)
x = layers.ReLU()(x)

x = layers.Dense(3*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(4*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(2*2*1024)(x)
x = layers.ReLU()(x)

x = layers.Reshape((2,2,1024))(x)

x = layers.Conv2DTranspose(1024, 1, padding='valid', kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2DTranspose(512, 2, padding='valid', kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2DTranspose(512, 2, padding='valid', kernel_initializer=initializer)(x)
#x = layers.BatchNormalization(axis=-1)(x)
x = layers.ReLU()(x)

x = layers.Conv2DTranspose(512, 2, padding='valid', kernel_initializer=initializer)(x)
#x = layers.BatchNormalization(axis=-1)(x)
x = layers.ReLU()(x)

#x = layers.Conv2DTranspose(256, 2, padding='valid', kernel_initializer=initializer)(x)
#x = layers.ReLU()(x)

x = layers.Conv2DTranspose(256, 2, padding='valid', kernel_initializer=initializer)(x)
#x = layers.BatchNormalization(axis=-1)(x)
x = layers.ReLU()(x)

x = layers.Conv2DTranspose(256, 2, padding='valid', kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2DTranspose(256, 3, padding='valid', kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2DTranspose(128, 2, padding='valid', kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2DTranspose(128, 2, padding='valid', kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2DTranspose(128, 2, padding='valid', kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2DTranspose(128, 2, padding='valid', kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2DTranspose(64, 2, padding='valid', kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.Conv2DTranspose(64, 2, padding='valid', kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

x = layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear')(x)

x = layers.Conv2DTranspose(8, 2, padding='valid', kernel_initializer=initializer)(x)
x = layers.ReLU()(x)

decoder_output = layers.Conv2DTranspose(3, 2, activation='sigmoid', padding='valid', kernel_initializer=initializer)(x)

decoder = keras.Model(latent_input, decoder_output, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 200)]             0         
                                                                 
 dense_2 (Dense)             (None, 400)               80400     
                                                                 
 re_lu_17 (ReLU)             (None, 400)               0         
                                                                 
 dense_3 (Dense)             (None, 600)               240600    
                                                                 
 re_lu_18 (ReLU)             (None, 600)               0         
                                                                 
 dense_4 (Dense)             (None, 800)               480800    
                                                                 
 re_lu_19 (ReLU)             (None, 800)               0   

In [8]:
decoder_out = decoder(encoder(encoder_input))
AE = keras.Model(encoder_input, decoder_out, name='AE')

AE.summary()

Model: "AE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 encoder (Functional)        (None, 200)               9280184   
                                                                 
 decoder (Functional)        (None, 32, 32, 3)         11084787  
                                                                 
Total params: 20364971 (77.69 MB)
Trainable params: 20364971 (77.69 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
K.clear_session()

In [9]:
#lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001, decay_steps=200, decay_rate=0.995)
optim = keras.optimizers.Adam(learning_rate = 0.001)                  # clipnorm=2.0
AE.compile(optimizer=optim,loss='mse')
history = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=20, steps_per_epoch=None)

Epoch 1/20
391/391 [==============================] - 63s 105ms/step - loss: 0.0621 - val_loss: 0.0619
Epoch 2/20
391/391 [==============================] - 35s 88ms/step - loss: 0.0621 - val_loss: 0.0618
Epoch 3/20
391/391 [==============================] - 35s 90ms/step - loss: 0.0603 - val_loss: 0.0500
Epoch 4/20
391/391 [==============================] - 35s 89ms/step - loss: 0.0415 - val_loss: 0.0376
Epoch 5/20
391/391 [==============================] - 35s 89ms/step - loss: 0.0371 - val_loss: 0.0341
Epoch 6/20
391/391 [==============================] - 35s 89ms/step - loss: 0.0336 - val_loss: 0.0332
Epoch 7/20
391/391 [==============================] - 35s 90ms/step - loss: 0.0333 - val_loss: 0.0323
Epoch 8/20
391/391 [==============================] - 35s 91ms/step - loss: 0.0311 - val_loss: 0.0307
Epoch 9/20
391/391 [==============================] - 35s 89ms/step - loss: 0.0306 - val_loss: 0.0305
Epoch 10/20
391/391 [==============================] - 35s 90ms/step - loss: 0.03

In [10]:
optim = keras.optimizers.Adam(learning_rate = 0.0005)                  # clipnorm=2.0
AE.compile(optimizer=optim,loss='mse')
history2 = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=15, steps_per_epoch=None)

Epoch 1/15
391/391 [==============================] - 47s 92ms/step - loss: 0.0255 - val_loss: 0.0252
Epoch 2/15
391/391 [==============================] - 35s 89ms/step - loss: 0.0250 - val_loss: 0.0251
Epoch 3/15
391/391 [==============================] - 35s 90ms/step - loss: 0.0248 - val_loss: 0.0249
Epoch 4/15
391/391 [==============================] - 35s 90ms/step - loss: 0.0247 - val_loss: 0.0247
Epoch 5/15
391/391 [==============================] - 35s 89ms/step - loss: 0.0245 - val_loss: 0.0246
Epoch 6/15
391/391 [==============================] - 35s 89ms/step - loss: 0.0243 - val_loss: 0.0244
Epoch 7/15
391/391 [==============================] - 35s 89ms/step - loss: 0.0242 - val_loss: 0.0244
Epoch 8/15
391/391 [==============================] - 35s 90ms/step - loss: 0.0240 - val_loss: 0.0241
Epoch 9/15
391/391 [==============================] - 35s 90ms/step - loss: 0.0235 - val_loss: 0.0235
Epoch 10/15
391/391 [==============================] - 35s 90ms/step - loss: 0.023

In [11]:
optim = keras.optimizers.Adam(learning_rate = 0.0001)                  # clipnorm=2.0
AE.compile(optimizer=optim,loss='mse')
history3 = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=10, steps_per_epoch=None)

Epoch 1/10
391/391 [==============================] - 48s 92ms/step - loss: 0.0221 - val_loss: 0.0225
Epoch 2/10
391/391 [==============================] - 35s 90ms/step - loss: 0.0220 - val_loss: 0.0225
Epoch 3/10
391/391 [==============================] - 35s 89ms/step - loss: 0.0219 - val_loss: 0.0226
Epoch 4/10
391/391 [==============================] - 36s 91ms/step - loss: 0.0219 - val_loss: 0.0225
Epoch 5/10
391/391 [==============================] - 35s 91ms/step - loss: 0.0218 - val_loss: 0.0225
Epoch 6/10
391/391 [==============================] - 35s 90ms/step - loss: 0.0218 - val_loss: 0.0225
Epoch 7/10
391/391 [==============================] - 35s 90ms/step - loss: 0.0217 - val_loss: 0.0225
Epoch 8/10
391/391 [==============================] - 35s 91ms/step - loss: 0.0217 - val_loss: 0.0225
Epoch 9/10
391/391 [==============================] - 36s 92ms/step - loss: 0.0217 - val_loss: 0.0225
Epoch 10/10
391/391 [==============================] - 36s 92ms/step - loss: 0.021

In [ ]:
optim = keras.optimizers.Adam(learning_rate = 0.00005)                  # clipnorm=2.0
AE.compile(optimizer=optim,loss='mse')
history4 = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=10, steps_per_epoch=None)

Epoch 1/10
391/391 [==============================] - 46s 90ms/step - loss: 0.0215 - val_loss: 0.0225
Epoch 2/10
391/391 [==============================] - 34s 87ms/step - loss: 0.0215 - val_loss: 0.0225
Epoch 3/10
391/391 [==============================] - 34s 88ms/step - loss: 0.0214 - val_loss: 0.0225
Epoch 4/10
391/391 [==============================] - 35s 89ms/step - loss: 0.0214 - val_loss: 0.0225
Epoch 5/10
391/391 [==============================] - 36s 91ms/step - loss: 0.0214 - val_loss: 0.0225
Epoch 6/10
179/391 [============>.................] - ETA: 17s - loss: 0.0214